# Notebook to organize Ingestion of XQ100 datasets

In [36]:
# imports
from astropy import units as u

from linetools.spectralline import AbsLine
from linetools import utils as ltu
from linetools.isgm.abscomponent import AbsComponent
from linetools.isgm import utils as ltiu


import pyigm
from pyigm.surveys import dlasurvey as dlasvy

## DLAs

### $N_{\rm HI}$ -- (Sanchez-Ramirez et al. 2016, MNRAS, 456, 4488)

In [3]:
xq100_statdla = dlasvy.DLASurvey.load_XQ100()
xq100_statdla

Loading QSOs file /Users/xavier/local/Python/pyigm/pyigm/data/DLA/XQ-100/XQ100_zpath.fit
Loaded


<IGMSurvey: nsys=41, type=DLA, ref=XQ-100, nsightlines=100>

### Chemical abundances -- Berg et al. 2016

    wget http://www.astro.uvic.ca/~trystynb/XSHOOTER/Metals/tb_XQ100_metals_to_json.dat
    Put into pyigm/pyigm/data/DLA/XQ-100/

#### Read Berg table

In [23]:
fil = pyigm.__path__[0]+'/data/DLA/XQ-100/tb_XQ100_metals_to_json.dat'
berg_tbl = Table.read(fil,format='ascii')
berg_tbl[0:5]

qso,ion,wl,zab,zem,ra,dec,vmin,vmax,filename,N,Nerr,flg,flg_analy,lambda,f
str10,str4,int64,float64,float64,str10,str9,int64,int64,str38,float64,float64,int64,int64,float64,float64
J0003-2603,OI,1302,3.39,4.125,00:03:22.8,-26:03:19,-60,70,J0003-2603/tb_J0003-2603_vis_cont.fits,15.1,0.02,3,-1,1302.168,0.04887
J0003-2603,MgI,2852,3.39,4.125,00:03:22.8,-26:03:19,-140,70,J0003-2603/tb_J0003-2603_nir_cont.fits,11.85,0.03,1,0,2852.964,1.81
J0003-2603,MgII,2796,3.39,4.125,00:03:22.8,-26:03:19,-220,200,J0003-2603/tb_J0003-2603_nir_cont.fits,13.91,0.02,2,0,2796.352,0.6123
J0003-2603,MgII,2803,3.39,4.125,00:03:22.8,-26:03:19,-200,170,J0003-2603/tb_J0003-2603_nir_cont.fits,14.04,0.01,2,0,2803.531,0.3054
J0003-2603,CaII,3934,3.39,4.125,00:03:22.8,-26:03:19,-20,50,J0003-2603/tb_J0003-2603_nir_cont.fits,11.81,0.0,3,0,3934.777,0.65


#### Systems

In [24]:
dlas = np.array(['{:s}_z{:0.3f}'.format(row['qso'],row['zab']) for row in berg_tbl])
unique_dlas = np.unique(dlas)
unique_dlas

array(['J0003-2603_z3.390', 'J0006-6208_z3.203', 'J0006-6208_z3.775',
       'J0034+1639_z3.755', 'J0034+1639_z4.252', 'J0034+1639_z4.284',
       'J0113-2803_z3.106', 'J0124+0044_z2.261', 'J0132+1341_z3.936',
       'J0134+0400_z3.692', 'J0134+0400_z3.772', 'J0234-1806_z3.693',
       'J0255+0048_z3.256', 'J0255+0048_z3.914', 'J0307-4945_z3.591',
       'J0307-4945_z4.466', 'J0415-4357_z3.808', 'J0424-2209_z2.982',
       'J0529-3552_z3.684', 'J0747+2739_z3.424', 'J0747+2739_z3.901',
       'J0800+1920_z3.946', 'J0818+0958_z3.306', 'J0835+0650_z3.955',
       'J0920+0725_z2.238', 'J0955-0130_z4.024', 'J1020+0922_z2.592',
       'J1024+1819_z2.298', 'J1057+1910_z3.373', 'J1058+1245_z3.432',
       'J1108+1209_z3.397', 'J1108+1209_z3.546', 'J1312+0841_z2.660',
       'J1421-0643_z3.449', 'J1517+0511_z2.688', 'J1552+1005_z3.601',
       'J1552+1005_z3.667', 'J1633+1411_z2.882', 'J1723+2243_z3.698',
       'J2239-0552_z4.080', 'J2344+0342_z3.220'], 
      dtype='|S17')

#### Loop on DLAs and generate

In [ ]:
for dla in unique_dlas:
    # Rows matching this DLA
    mti = np.where(dlas == dla)[0]
    # Generate abslines
    abslines = []
    for ii in mti:
        row = berg_tbl[ii]
        if row['ion'] == 'TiII':
            print('Skipping TiII')
            continue
        #
        trans = '{:s} {:d}'.format(row['ion'],row['wl'])
        print('DLA={:s}, trans={:s}'.format(dla,trans))
        aline = AbsLine(trans)
        # Coord, zem, etc. attrib
        aline.attrib['coord'] = ltu.radec_to_coord((row['ra'],row['dec']))
        aline.attrib['z'] = row['zab']
        aline.attrib['logN'] = row['N']
        aline.attrib['sig_logN'] = row['Nerr']
        aline.attrib['flag_N'] = row['flg']
        # Analysis
        aline.analy['vlim'] = [row['vmin'], row['vmax']]*u.km/u.s
        aline.analy['do_analysis'] = row['flg_analy']
        aline.analy['spec'] = row['filename']
        # Checks
        np.testing.assert_allclose(aline.wrest.value, row['lambda'], rtol=1e-4)
        try:
            np.testing.assert_allclose(aline.data['f'], row['f'], rtol=1e-4)
        except AssertionError:
            print("f-values do not closely match for trans={:s}".format(trans))
            print("f_lt={:g}, f_Berg={:g}".format(aline.data['f'], row['f']))
        # Add
        abslines.append(aline)
    # Generate components
    comps = ltiu.build_components_from_abslines(abslines, chk_vel=False)
    xdb.set_trace()

DLA=J0003-2603_z3.390, trans=OI 1302
f-values do not closely match for trans=OI 1302
f_lt=0.048, f_Berg=0.04887
DLA=J0003-2603_z3.390, trans=MgI 2852
f-values do not closely match for trans=MgI 2852
f_lt=1.73, f_Berg=1.81
DLA=J0003-2603_z3.390, trans=MgII 2796
f-values do not closely match for trans=MgII 2796
f_lt=0.6155, f_Berg=0.6123
DLA=J0003-2603_z3.390, trans=MgII 2803
f-values do not closely match for trans=MgII 2803
f_lt=0.3058, f_Berg=0.3054
DLA=J0003-2603_z3.390, trans=CaII 3934
DLA=J0003-2603_z3.390, trans=CaII 3969
DLA=J0003-2603_z3.390, trans=SiII 1304
f-values do not closely match for trans=SiII 1304
f_lt=0.0863, f_Berg=0.094
DLA=J0003-2603_z3.390, trans=SiII 1526
DLA=J0003-2603_z3.390, trans=SiII 1808
f-values do not closely match for trans=SiII 1808
f_lt=0.00208, f_Berg=0.002186
DLA=J0003-2603_z3.390, trans=CrII 2056
f-values do not closely match for trans=CrII 2056
f_lt=0.103, f_Berg=0.105
DLA=J0003-2603_z3.390, trans=CrII 2062
f-values do not closely match for trans=Cr

In [17]:
abslines

[<AbsLine: OI 1302, wrest=1302.1685 Angstrom>,
 <AbsLine: MgI 2852, wrest=2852.9642 Angstrom>,
 <AbsLine: MgII 2796, wrest=2796.3543 Angstrom>,
 <AbsLine: MgII 2803, wrest=2803.5315 Angstrom>,
 <AbsLine: CaII 3934, wrest=3934.7770 Angstrom>]